In [1]:
import dask.dataframe as dd
from dask.distributed import Client, LocalCluster
import xgboost

In [2]:
cluster = LocalCluster()
client = Client(cluster)

In [3]:
NYC_TAXI_DATA_PARQUET = '/home/spotter/Documents/job/coiled_assessment/data/parquet/nyc-taxi'

In [4]:
use_cols = [
    'tpep_pickup_datetime',
    'passenger_count',
    'trip_distance',
    'fare_amount',
    'tip_amount',
    'total_amount']
df = dd.read_parquet(NYC_TAXI_DATA_PARQUET, columns=use_cols)
df.head()

,tpep_pickup_datetime,passenger_count,trip_distance,fare_amount,tip_amount,total_amount
0,2019-01-01 00:46:40,1.0,1.5,7.0,1.65,9.95
1,2019-01-01 00:59:47,1.0,2.6,14.0,1.00,16.30
7,2019-01-01 00:21:28,1.0,1.3,6.5,1.25,9.05
8,2019-01-01 00:32:01,1.0,3.7,13.5,3.70,18.50
9,2019-01-01 00:57:32,2.0,2.1,10.0,1.70,13.00


In [5]:
df['tip_frac'] = df['tip_amount'] / df['total_amount']
df['pickup_hour'] = dd.to_datetime(df['tpep_pickup_datetime']).dt.hour
df.head()

,tpep_pickup_datetime,passenger_count,trip_distance,fare_amount,tip_amount,total_amount,tip_frac,pickup_hour
0,2019-01-01 00:46:40,1.0,1.5,7.0,1.65,9.95,0.165829,0
1,2019-01-01 00:59:47,1.0,2.6,14.0,1.00,16.30,0.061350,0
7,2019-01-01 00:21:28,1.0,1.3,6.5,1.25,9.05,0.138122,0
8,2019-01-01 00:32:01,1.0,3.7,13.5,3.70,18.50,0.200000,0
9,2019-01-01 00:57:32,2.0,2.1,10.0,1.70,13.00,0.130769,0


In [6]:
train, test = df.random_split([0.8, 0.2])
train_labels = train.tip_frac
test_labels = test.tip_frac

In [7]:
del train['tip_frac']
del test['tip_frac']

del train['tpep_pickup_datetime']
del test['tpep_pickup_datetime']

In [8]:
dtrain = xgboost.dask.DaskDMatrix(client, train, train_labels)

In [9]:
params = {
    'verbosity': 2,
    'tree_method': 'hist',
    'objective': 'reg:squarederror'
}
output = xgboost.dask.train(
    client,
    params,
    dtrain,
    evals=[(dtrain, 'train')]
)

[20:36:18] task [xgboost.dask]:tcp://127.0.0.1:43095 got new rank 0
[20:36:18] task [xgboost.dask]:tcp://127.0.0.1:35245 got new rank 1
[20:36:18] task [xgboost.dask]:tcp://127.0.0.1:43569 got new rank 2
[20:36:18] task [xgboost.dask]:tcp://127.0.0.1:44197 got new rank 3
distributed.worker - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker.html#memtrim for more information. -- Unmanaged memory: 2.59 GiB -- Worker memory limit: 3.88 GiB
distributed.worker - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker.html#memtrim for more information. -- Unmanaged memory: 2.76 GiB -- Worker memory limit: 3.88 GiB
distributed.worker - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distri

distributed.worker - WARNING - Worker is at 53% memory usage. Resuming worker. Process memory: 2.09 GiB -- Worker memory limit: 3.88 GiB
[20:36:23] INFO: ../src/gbm/gbtree.cc:177: Tree method is selected to be 'hist', which uses a single updater grow_quantile_histmaker.
[20:36:23] INFO: ../src/gbm/gbtree.cc:177: Tree method is selected to be 'hist', which uses a single updater grow_quantile_histmaker.
[20:36:23] INFO: ../src/gbm/gbtree.cc:177: Tree method is selected to be 'hist', which uses a single updater grow_quantile_histmaker.
[20:36:23] INFO: ../src/gbm/gbtree.cc:177: Tree method is selected to be 'hist', which uses a single updater grow_quantile_histmaker.
[20:36:24] INFO: ../src/gbm/gbtree.cc:177: Tree method is selected to be 'hist', which uses a single updater grow_quantile_histmaker.
[20:36:24] INFO: ../src/gbm/gbtree.cc:177: Tree method is selected to be 'hist', which uses a single updater grow_quantile_histmaker.
[20:36:24] INFO: ../src/gbm/gbtree.cc:177: Tree method is s

distributed.worker - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker.html#memtrim for more information. -- Unmanaged memory: 3.18 GiB -- Worker memory limit: 3.88 GiB
distributed.worker - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker.html#memtrim for more information. -- Unmanaged memory: 3.37 GiB -- Worker memory limit: 3.88 GiB
distributed.worker - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker.html#memtrim for more information. -- Unmanaged memory: 3.37 GiB -- Worker memory limit: 3.88 GiB
distributed.worker - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see 

[0]	train-rmse:0.24783


distributed.worker - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker.html#memtrim for more information. -- Unmanaged memory: 2.73 GiB -- Worker memory limit: 3.88 GiB
distributed.worker - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker.html#memtrim for more information. -- Unmanaged memory: 3.44 GiB -- Worker memory limit: 3.88 GiB
distributed.worker - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker.html#memtrim for more information. -- Unmanaged memory: 3.26 GiB -- Worker memory limit: 3.88 GiB
distributed.worker - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see 

...

[8]	train-rmse:0.01622


distributed.worker - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker.html#memtrim for more information. -- Unmanaged memory: 2.74 GiB -- Worker memory limit: 3.88 GiB
distributed.worker - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker.html#memtrim for more information. -- Unmanaged memory: 3.44 GiB -- Worker memory limit: 3.88 GiB
distributed.worker - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker.html#memtrim for more information. -- Unmanaged memory: 3.26 GiB -- Worker memory limit: 3.88 GiB
distributed.worker - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see 

distributed.worker - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker.html#memtrim for more information. -- Unmanaged memory: 2.74 GiB -- Worker memory limit: 3.88 GiB
distributed.worker - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker.html#memtrim for more information. -- Unmanaged memory: 3.44 GiB -- Worker memory limit: 3.88 GiB
distributed.worker - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker.html#memtrim for more information. -- Unmanaged memory: 3.26 GiB -- Worker memory limit: 3.88 GiB
distributed.worker - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see 

[9]	train-rmse:0.01226


distributed.worker - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker.html#memtrim for more information. -- Unmanaged memory: 2.74 GiB -- Worker memory limit: 3.88 GiB
distributed.worker - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker.html#memtrim for more information. -- Unmanaged memory: 3.33 GiB -- Worker memory limit: 3.88 GiB
distributed.worker - WARNING - Worker is at 58% memory usage. Resuming worker. Process memory: 2.27 GiB -- Worker memory limit: 3.88 GiB
distributed.worker - WARNING - Worker is at 59% memory usage. Resuming worker. Process memory: 2.31 GiB -- Worker memory limit: 3.88 GiB


In [10]:
prediction = xgboost.dask.predict(client, output, test)
rmse = ((prediction - test_labels) ** 2).mean() ** 0.5
rmse = rmse.compute()
print(f'RMSE of model against hold out data {rmse}')

distributed.worker - WARNING - Worker is at 37% memory usage. Resuming worker. Process memory: 1.45 GiB -- Worker memory limit: 3.88 GiB


RMSE of model against hold out data 0.060184543349072146


In [11]:
client.shutdown()